In [273]:
from math import log
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
from pprint import pprint
from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import scale
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import Binarizer 

In [288]:
# data = pd.read_csv("WA_Fn-UseC_-Telco-Customer-Churn.csv")
# data.drop('customerID',axis=1,inplace=True)
# data= pd.DataFrame(data)
# print(data)
# data['TotalCharges'] = data["TotalCharges"].replace(" ",0)
# data['TotalCharges']= pd.to_numeric(data['TotalCharges'])
# data['TotalCharges'] = data["TotalCharges"].replace(0,data['TotalCharges'].mean())
# data['TotalCharges']
# target = data["Churn"]
# print ("\nMissing values :  ", data.isnull().sum().values.sum())
# print ("\nUnique values :  \n",data.nunique())

# cat = data.nunique()[data.nunique() < 6].keys().tolist()
# target_col = ["Churn"]
# cat = [x for x in cat if x not in target_col]
# num_cols   = [x for x in data.columns if x not in cat + target_col ]
# bin_cols = data.nunique()[data.nunique() == 2].keys().tolist()
# bin_cols = [x for x in bin_cols if x not in target_col]
# multi_cols = [x for x in cat if x not in bin_cols]

# #binary encoding
# le = preprocessing.LabelEncoder()
# for i in bin_cols:
#     data[i] = le.fit_transform(data[i])
#     print(data[i])

# #string -> encoding
# le = preprocessing.LabelEncoder()
# for i in multi_cols:
#     data[i] = le.fit_transform(data[i])
#     print(data[i])
    
# #Binarization
# for i in range(len(num_cols)):
#     binarizer= Binarizer(data[num_cols[i]].median())
#     data[num_cols[i]] = binarizer.fit_transform(data[num_cols[i]].values.reshape(1,-1)).reshape(-1,1)

In [289]:
 data = pd.DataFrame({"toothed":["True","True","True","False","True","True","True","True","True","False"],"breathes":["True","True","True","True","True","True","False","True","True","True"],"legs":["True","True","False","True","True","True","False","False","True","True"],"species":["Mammal","Mammal","Reptile","Mammal","Mammal","Mammal","Reptile","Reptile","Mammal","Reptile"]})
# data = pd.read_csv("tennis.csv")
# data = pd.read_csv("hd.csv")
# print(data)
features= data.columns[:-1]
print(features)
target = data.columns[features.size]
print(target)
columns=data.columns
# data

# y=data[target]
# data.drop(target,axis=1,inplace=True)
# x=np.array(data)
# x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=.2)
# print(x_train,y_train,x_test,y_test)

Index(['toothed', 'breathes', 'legs'], dtype='object')
species


In [290]:
def entropy(column):
    elements,counts = np.unique(column,return_counts = True)
    entropy = 0
    for i in range(elements.size):
        entropy += counts[i]/np.sum(counts)*log(counts[i]/np.sum(counts),2)
    entropy= -entropy
    return entropy

In [291]:
def InfoGain(data,attribute_name):
    attribute= data.get(attribute_name)
    total_entropy = entropy(data.get(target))
    elements,counts = np.unique(attribute,return_counts = True)
    split_columns={}
    for i in range(elements.size):
        split_columns[i] = data[attribute == elements[i]]  
    weighted_Entropy = 0
    for i in range(elements.size):
        weighted_Entropy += (counts[i]/np.sum(counts))*entropy(split_columns[i].get(target))
    Info_gain = total_entropy - weighted_Entropy 
    return Info_gain

In [292]:
def PLURALITY_VALUE(example):
    elements,count = np.unique(example[target],return_counts=True)
    selected_index =  np.argmax(count)
#     print(np.argmax(count))
#     print(np.unique(example[target])[selected_index])
    return np.unique(example[target])[selected_index]

In [293]:
def decision_tree_learning(examples, attributes, parent_examples, depth, tree):
    
# if examples is empty then return PLURALITY_VALUE(parent_examples)
    if depth == 0:
        tree = PLURALITY_VALUE(examples)
        return tree
    
    if examples.size==0 :
        return PLURALITY_VALUE(parent_examples)
    
# else if all examples have same classification then return the classification
    elif np.unique(examples[target]).size < 2:
        return np.unique(examples[target])[0]
    
# else if attributes is empty then return PLURALITY_VALUE(examples)
    elif attributes.size == 0:
        return PLURALITY_VALUE(examples)
    
    else:
        parent_examples = PLURALITY_VALUE(examples)
        
        InfoGain_values = np.empty((0, attributes.size))
        for feature in attributes:
            InfoGain_values = np.append(InfoGain_values,InfoGain(examples,feature))
        
        selected_column = examples.get(attributes[np.argmax(InfoGain_values)])
        best_attribute = attributes[np.argmax(InfoGain_values)]
        
        tree = {best_attribute:{}}
       
        featuresNew = np.empty((0, attributes.size-1))
        for i in range(attributes.size):
            if(attributes[i]!=best_attribute):
                featuresNew = np.append(featuresNew,attributes[i])
                
        elements,counts=np.unique(selected_column,return_counts=True)
        
        for i in range((np.unique(selected_column)).size):
            value = np.unique(selected_column)[i]
            exs = examples.get(examples[best_attribute] == value)
            subtree = decision_tree_learning(exs,featuresNew,parent_examples, depth-1, tree)
            tree[best_attribute][value] = subtree
        return tree

In [294]:
tree={}
tree= decision_tree_learning(data,features, None,1, tree )
pprint(tree)

{'legs': {'False': 'Reptile', 'True': 'Mammal'}}


In [285]:
def Normalize(vector):
    return vector / np.linalg.norm(vector)

In [31]:
def AdaBoost(examples,L,K):
#     examples, set of N labeled examples (x , y ), ... , (x , y )
#     L, a learning algorithm
#     K, the number of hypotheses in the ensemble
    w = np.full((1,len(examples)), 1/len(examples)) #w, a vector of N example weights, initially 1 N
    h = np.full((1,K), 0) #h, a vector of K hypotheses
    z = np.full((1,K), 0) #z, a vector of K hypothesis weights
    
    for k in K:
        data = Resample(examples, w)
        h[k] = L(data)
        error = 0
        for j in len(examples):
            if h[k].get(xj)!=yj :
                error = error + w[j]
        if error > 0.5 :
            continue
        for j in len(examples):
            if h[k](xj) = yj:
                w[j] = w[j] * error/(1 − error)
        w = Normalize(w)
        z[k] = log ((1-error)/error)
        
    return Weighted_Majority(h,z)             
    

(6, 4) (6,) (2, 4) (2,)
